In [2]:
import os
import re
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [20]:
#THIS IS A PERMANENT CELL
model_area = "NSSA"

rolling_average = False

summation_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Summation.csv"
dwf_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\DWF_Specs.csv"
parameter_script = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Read_Parameters.py"
outfall_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Outfall_Summary.csv"

calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\SYSTEM_ASSESSMENT\DATA\RAINFALL\NSSA_Rainfall_Data_PDT.dfs0"

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

In [21]:
#THIS IS A PERMANENT CELL
import subprocess

use_accumulation = True #Must be string to
model = r'J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\NSSA_Base.mdb'

bat_file_path = output_folder + '\\Read_Parameters.bat'
bat_file = open(bat_file_path, "w")
bat_file.write('"' + parameter_script + '" "' + output_folder + '" "' + model + '" ' + str(use_accumulation))
bat_file.close()

subprocess.call([bat_file_path])


0

In [5]:
#THIS IS A PERMANENT CELL
res_types = ['Mixed','ResHD','ResLD']
ici_types = ['Commercial','Industrial','Institutional']
ww_types = res_types + ici_types

all_types = []
for res_type in res_types:
    all_types.append([res_type,'Population','pe','m3/pe/d'])
for ici_type in ici_types:
    all_types.append([ici_type,'Area','ha','m3/ha/d'])
all_types.append(['Baseflow','','',''])
all_types.append(['Total','','',''])
all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
all_types.set_index('Load_Type',inplace=True)

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

dwf_specs = pd.read_csv(dwf_csv)
dwf_specs.set_index('Zone',inplace=True)

,Mixed_Rate,ResHD_Rate,ResLD_Rate,Commercial_Rate,Industrial_Rate,Institutional_Rate,Mixed_Population,ResHD_Population,ResLD_Population,Commercial_Area,...,Industrial_Area_Upstream,Institutional_Area_Upstream,Mixed_WaterLoad_Upstream,ResHD_WaterLoad_Upstream,ResLD_WaterLoad_Upstream,Commercial_WaterLoad_Upstream,Industrial_WaterLoad_Upstream,Institutional_WaterLoad_Upstream,Baseflow_WaterLoad_Upstream,Total_WaterLoad_Upstream
Zone,,,,,,,,,,,,,,,,,,,,,
FA2,0.185,0.185,0.185,10.0,10.0,10.0,124.697553,1376.568852,19538.388103,4.578282,...,0.0,14.067279,23.069047,254.665238,3614.601799,45.782818,0.0,140.672793,2557.439,6636.230695
GE1,0.250,0.250,0.250,85.0,85.0,85.0,0.000000,91.376044,1544.746896,1.905471,...,0.0,0.000000,0.000000,22.844011,386.186724,161.965000,0.0,0.000000,25.920,596.915735


In [7]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]
report_text.fillna('This section was left blank.', inplace=True)

wwf_specs = pd.read_excel(calibration_sheet,sheet_name="Parameters_WWF")
wwf_specs.set_index('Location',inplace=True)
wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')

wwf_stats_specs

,Description,DF,DF_Col,Unit,Table
0,Accumulated rainfall,NaN,NaN,mm,1
1,Drainage area,wwf_specs,Drainage area (ha),ha,1
2,Number sanitary,wwf_specs,Number sanitary,-,1
3,Number stormwater,wwf_specs,Number combined,-,1
4,Number combined,wwf_specs,Number stormwater,-,1
5,Area sanitary,wwf_specs,Area sanitary (ha),ha,1
6,Area combined,wwf_specs,Area combined (ha),ha,1
7,Area stormwater,wwf_specs,Area stormwater (ha),ha,1
8,Flat impervious area,wwf_specs,Flat Impervious area (ha),ha,2
9,Steep impervious area,wwf_specs,Steep Impervious area (ha),ha,2


In [9]:
wwf_specs.to_csv(r'J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\WWF_Specs_Jupyter.csv')


,Node,Level Type
241,43987,Link(DS)
253,54852,Link(DS)
254,54850,Link(DS)


In [14]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)

In [15]:
for ds_level_pipe in ds_level_pipes:
    print(ds_level_pipe)

54850
54852
43987


In [16]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'HL14_20180614_20190403_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

,HL14_Flow_L,HL14_Level
2018-06-14 00:00:00,105.150002,-0.659200
2018-06-14 00:05:00,106.050003,-0.657905
2018-06-14 00:10:00,108.650002,-0.654010
2018-06-14 00:15:00,110.349998,-0.650690
2018-06-14 00:20:00,110.150002,-0.650770
...,...,...
2019-04-03 23:35:00,109.550003,-0.649750
2019-04-03 23:40:00,110.150002,-0.648520
2019-04-03 23:45:00,109.849998,-0.648040
2019-04-03 23:50:00,108.699997,-0.649165


In [21]:
f = 'HL14_20180614_20190403_5min.dfs0'

ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
gauge_id = re.split('_|.',f)[0]
ts['Gauge'] = gauge_id
ts.rename(columns={ts.columns[1]:'Flow'},inplace=True)
if len(ts.columns)>6:
    ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
else:
    ts['Level'] = np.nan
ts = ts[['Gauge','Flow','Level']]


In [17]:
#Import rain gauges

rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            gauge_id = re.split(r'_|[.]',f)[0]
            ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
            ts['Gauge'] = gauge_id
            gauge_ids.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if len(ts.columns)>5:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
            else:
                ts['Level'] = np.nan
            ts = ts[['Gauge','Flow','Level']]
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
            
measured['Seconds'] = measured.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
measured['Volume'] = measured.Flow * measured.Seconds / 1000
            

            
        

In [385]:
rainfall

,VW14,DN53,VN52,VW49,DN25,DN82,DN54,DN65,VW67,DN72
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2022-01-17 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-17 23:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
list(di.CreateTimeSeriesData(2))

[12.31151,
 12.31077,
 12.31001,
 12.30925,
 12.30849,
 12.30771,
 12.30694,
 12.30617,
 12.30546,
 12.30481,
 12.30417,
 12.30355,
 12.30291,
 12.30227,
 12.30163,
 12.30099,
 12.30033,
 12.29967,
 12.29901,
 12.29837,
 12.29785,
 12.29749,
 12.29718,
 12.29689,
 12.2966,
 12.29632,
 12.29603,
 12.29574,
 12.29545,
 12.29515,
 12.29486,
 12.29457,
 12.29432,
 12.29412,
 12.29393,
 12.29374,
 12.29356,
 12.29338,
 12.29319,
 12.293,
 12.29281,
 12.29262,
 12.29244,
 12.29226,
 12.29214,
 12.29211,
 12.2921,
 12.29211,
 12.29212,
 12.29212,
 12.29213,
 12.29214,
 12.29215,
 12.29216,
 12.29217,
 12.29219,
 12.29225,
 12.29237,
 12.29251,
 12.29266,
 12.29282,
 12.29297,
 12.29313,
 12.29328,
 12.29344,
 12.29359,
 12.29374,
 12.29393,
 12.2944,
 12.29522,
 12.29615,
 12.29709,
 12.29806,
 12.29901,
 12.29994,
 12.30088,
 12.3018,
 12.3027,
 12.30361,
 12.30452,
 12.30559,
 12.30682,
 12.30808,
 12.30933,
 12.31058,
 12.31182,
 12.31304,
 12.31424,
 12.31543,
 12.31661,
 12.31778,
 12.31

In [18]:
results = []
first_level = True
first_flow = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches
        nodes = resultdata.resultData.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

#         missing_muids = set()

#         #Check for missing MUIDs in the result file
#         muid_checks = set()

#         rid_checks = set()
#         for i, reach in enumerate(reaches):
#             rid_checks.add(reach.Id[:(-1 * len(str(i)) - 1)])
#         for rid in network_import_specs["MUID_Extended"].values:
#             if not rid in rid_checks:
#                 muid_checks.add(rid)
#         if len(muid_checks)>0:
#             print('In Network_Import_Specs but not in re: ' + str(muid_checks))
#             break

        for i, node in enumerate(nodes):
            
            rid = node.Id
            if rid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                for di in node.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        values = list(di.CreateTimeSeriesData(0))
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            levelpoints = (gridpoints + 1)/2

            if rid in us_level_pipes or rid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))
                print (gridpoints)
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        if rid in us_level_pipes: 
                            values = list(di.CreateTimeSeriesData(0))
                        else:
                            values = list(di.CreateTimeSeriesData(levelpoints - 1))
                            
                        
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
            
        for i, reach in enumerate(reaches):
            
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            if 'Weir:' in rid:
                rid = rid[5:]
            if 'Orifice:' in rid:
                rid = rid[8:]
            gridpoints = reach.GridPoints.Count
            if rid in flow_pipes or rid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'discharge':

                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
                        flow_df['MUID'] = rid
                        flow_df['DateTimeRef'] = flow_df.index
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000
                        
                        flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                        flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                        flow_df.drop(columns=['Seconds'],inplace=True)

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

                        
                        if first_flow == True:
                            flow_df_all = flow_df.copy()

                        else:
                            flow_df_all = pd.concat([flow_df_all,flow_df])
                        first_flow = False

        first_round = False
        
#Summation
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

flow_df_all = pd.concat([flow_df_all,df_result_sum])
# flow_df_all['Seconds'] = flow_df_all.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
# flow_df_all['Volume'] = flow_df_all.Discharge * flow_df_all.Seconds / 1000


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-11-08 14:39:27.863664
Importing node 10497 at 2022-11-08 14:39:27.863664
Importing node 10498 at 2022-11-08 14:39:27.868696
Importing node 1_119 at 2022-11-08 14:39:27.871699
Importing node 6071 at 2022-11-08 14:39:27.875727
Importing node 6124 at 2022-11-08 14:39:27.879731
Importing node 6819 at 2022-11-08 14:39:27.883734
Importing node 6822 at 2022-11-08 14:39:27.886737
Importing node 7133 at 2022-11-08 14:39:27.890767
Importing node 7169 at 2022-11-08 14:39:27.893769
Importing node 7317 at 2022-11-08 14:39:27.897800
Importing node 9488 at 2022-11-08 14:39:27.900801
Importing node 9525 at 2022-11-08 14:39:27.903803
Importing node 9551 at 2022-11-08 14:39:27.906829
Importing node 9561 at 2022-11-08 14:39:27.909832
Importing node 9600 at 2022-11-08 14:39:27.913863
Importing node 9619 at 2022-11-08 14:39:27.916864
Importing node 9624 at 2022-11-08 14:39:27.919867
Importing node 9629 at 2022-11-08 14:39:27.922870
Importing node

Importing network NSSA_DWF_Cal_2018-08-25_Base.res1d at 2022-11-08 14:39:29.825483
Importing node 10497 at 2022-11-08 14:39:29.825483
Importing node 10498 at 2022-11-08 14:39:29.829485
Importing node 1_119 at 2022-11-08 14:39:29.834511
Importing node 6071 at 2022-11-08 14:39:29.839516
Importing node 6124 at 2022-11-08 14:39:29.843520
Importing node 6819 at 2022-11-08 14:39:29.848524
Importing node 6822 at 2022-11-08 14:39:29.852558
Importing node 7133 at 2022-11-08 14:39:29.856561
Importing node 7169 at 2022-11-08 14:39:29.860592
Importing node 7317 at 2022-11-08 14:39:29.865610
Importing node 9488 at 2022-11-08 14:39:29.869624
Importing node 9525 at 2022-11-08 14:39:29.873656
Importing node 9551 at 2022-11-08 14:39:29.877658
Importing node 9561 at 2022-11-08 14:39:29.881662
Importing node 9600 at 2022-11-08 14:39:29.885665
Importing node 9619 at 2022-11-08 14:39:29.890694
Importing node 9624 at 2022-11-08 14:39:29.894698
Importing node 9629 at 2022-11-08 14:39:29.898724
Importing node

Importing network NSSA_WWF_Cal_2018-11-25_Base.res1d at 2022-11-08 14:39:31.789199
Importing node 10497 at 2022-11-08 14:39:31.789199
Importing node 10498 at 2022-11-08 14:39:31.794227
Importing node 1_119 at 2022-11-08 14:39:31.798231
Importing node 6071 at 2022-11-08 14:39:31.803235
Importing node 6124 at 2022-11-08 14:39:31.808240
Importing node 6819 at 2022-11-08 14:39:31.813270
Importing node 6822 at 2022-11-08 14:39:31.816273
Importing node 7133 at 2022-11-08 14:39:31.821302
Importing node 7169 at 2022-11-08 14:39:31.825338
Importing node 7317 at 2022-11-08 14:39:31.829340
Importing node 9488 at 2022-11-08 14:39:31.833371
Importing node 9525 at 2022-11-08 14:39:31.837373
Importing node 9551 at 2022-11-08 14:39:31.843378
Importing node 9561 at 2022-11-08 14:39:31.847382
Importing node 9600 at 2022-11-08 14:39:31.851414
Importing node 9619 at 2022-11-08 14:39:31.855417
Importing node 9624 at 2022-11-08 14:39:31.859445
Importing node 9629 at 2022-11-08 14:39:31.863448
Importing node

Importing pipe Weir:MKO_Overflow-1635 discharge at 2022-11-08 14:39:33.128819
Importing network NSSA_WWF_Cal_2019-01-02_Base.res1d at 2022-11-08 14:39:33.696212
Importing node 10497 at 2022-11-08 14:39:33.696212
Importing node 10498 at 2022-11-08 14:39:33.702240
Importing node 1_119 at 2022-11-08 14:39:33.706272
Importing node 6071 at 2022-11-08 14:39:33.712277
Importing node 6124 at 2022-11-08 14:39:33.717304
Importing node 6819 at 2022-11-08 14:39:33.722309
Importing node 6822 at 2022-11-08 14:39:33.727313
Importing node 7133 at 2022-11-08 14:39:33.731344
Importing node 7169 at 2022-11-08 14:39:33.736349
Importing node 7317 at 2022-11-08 14:39:33.740379
Importing node 9488 at 2022-11-08 14:39:33.745407
Importing node 9525 at 2022-11-08 14:39:33.750410
Importing node 9551 at 2022-11-08 14:39:33.755439
Importing node 9561 at 2022-11-08 14:39:33.760443
Importing node 9600 at 2022-11-08 14:39:33.765448
Importing node 9619 at 2022-11-08 14:39:33.770543
Importing node 9624 at 2022-11-08 14

Importing pipe 54850-1537 discharge at 2022-11-08 14:39:35.250818
Importing pipe 54852-1539 discharge at 2022-11-08 14:39:35.258825
Importing pipe 55199-1545 discharge at 2022-11-08 14:39:35.267833
Importing pipe 56296-1593 discharge at 2022-11-08 14:39:35.276841
Importing pipe 56393-1597 discharge at 2022-11-08 14:39:35.285849
Importing pipe 56394_3-1600 discharge at 2022-11-08 14:39:35.293857
Importing pipe 56401-1605 discharge at 2022-11-08 14:39:35.302865
Importing pipe 56405-1606 discharge at 2022-11-08 14:39:35.310912
Importing pipe Link_15-1612 discharge at 2022-11-08 14:39:35.319943
Importing pipe Weir:MKO_Overflow-1635 discharge at 2022-11-08 14:39:35.327950
Importing network NSSA_WWF_Cal_2020-01-29_Base.res1d at 2022-11-08 14:39:36.003655
Importing node 10497 at 2022-11-08 14:39:36.003655
Importing node 10498 at 2022-11-08 14:39:36.010694
Importing node 1_119 at 2022-11-08 14:39:36.015699
Importing node 6071 at 2022-11-08 14:39:36.021726
Importing node 6124 at 2022-11-08 14:3

In [49]:
flow_df_all

,ResultFile,MUID,DateTimeRef,Discharge,Volume
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:00:00,1.646884e+01,4.940652e+00
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:05:00,1.617527e+01,4.852581e+00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:10:00,1.588190e+01,4.764570e+00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:15:00,1.558910e+01,4.676730e+00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,2018-07-26 00:20:00,1.529686e+01,4.589058e+00
...,...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:40:00,1.160301e-09,3.480903e-10
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:45:00,1.884038e+02,5.652114e+01
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:50:00,1.887150e+02,5.661450e+01
2020-02-01 23:55:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:55:00,1.891300e+02,5.673900e+01


In [422]:
flow_df_all['Seconds'] = flow_df_all.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
# flow_df_all['Seconds'] = flow_df_all.groupby(['ResultFile','MUID']).index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
flow_df_all['Volume'] = flow_df_all.Discharge * flow_df_all.Seconds / 1000
flow_df_all

AttributeError: 'DataFrameGroupBy' object has no attribute 'index'

In [19]:
df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum'})
df_result_sum.reset_index(inplace=True)
df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)
df_result_sum

,ResultFile,MUID,DateTimeRef,Discharge
DateTimeRef,,,,
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,Lynn-Total,2018-07-26 00:00:00,1.849718e+02
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,Lynn-Total,2018-07-26 00:05:00,0.000000e+00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,Lynn-Total,2018-07-26 00:10:00,0.000000e+00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,Lynn-Total,2018-07-26 00:15:00,0.000000e+00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,Lynn-Total,2018-07-26 00:20:00,1.868968e+02
...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:40:00,1.160301e-09
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:45:00,1.884038e+02
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Lynn-Total,2020-02-01 23:50:00,1.887150e+02


In [183]:
df_result_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18450 entries, 0 to 18449
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ResultFile   18450 non-null  object        
 1   MUID         18450 non-null  object        
 2   DateTimeRef  18450 non-null  datetime64[ns]
 3   Discharge    18450 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 576.7+ KB


In [184]:
flow_df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 369000 entries, 2018-07-26 00:00:00 to 2020-02-02 00:00:00
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   ResultFile   369000 non-null  object        
 1   MUID         369000 non-null  object        
 2   Discharge    369000 non-null  float64       
 3   DateTimeRef  369000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 14.1+ MB


In [ ]:
# import plotly
# a = np.random.normal(0,1,100)
# b = np.random.normal(-2,5,100)

# c = np.random.normal(0,1,100)
# d = np.random.normal(-2,5,100)

# fig = plotly.tools.make_subplots(rows=2,cols=1)

# trace_rm1 = go.Histogram(x = a, opacity = 0.75, name = 'malignant')
# trace_rm2 = go.Histogram(x = b, opacity = 0.75, name = 'benign')
# fig.append_trace(go.Histogram(x = a, opacity = 0.75, name = 'benign'),1,1)
# fig.append_trace(go.Histogram(x = b, opacity = 0.75, name = 'malignant'),1,1)
# fig.append_trace(go.Histogram(x = c, opacity = 0.75, name = 'benign'),2,1)
# fig.append_trace(go.Histogram(x = d, opacity = 0.75, name = 'malignant'),2,1)
# fig.layout.update(go.Layout(barmode = 'overlay',))

# fig.show()

In [112]:
flow_df_all

,ResultFile,MUID,Discharge,DateTimeRef
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.468840,2018-07-26 00:00:00
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.175270,2018-07-26 00:05:00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.881900,2018-07-26 00:10:00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.589100,2018-07-26 00:15:00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.296860,2018-07-26 00:20:00
...,...,...,...,...
18445,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.503012,2020-02-01 23:40:00
18446,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.694494,2020-02-01 23:45:00
18447,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,126.869387,2020-02-01 23:50:00
18448,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,125.155301,2020-02-01 23:55:00


In [45]:
effluent_types = ['Com','st','in','Henrik','Logan']
for i, effluent_type in enumerate(effluent_types):
    if i == 0:
        effluent_type_string = effluent_type
    elif i == len(effluent_types)-1:
        effluent_type_string += ' and ' + effluent_type
    else:
        effluent_type_string += ', ' + effluent_type

effluent_type_string

'Com, st, in, Henrik and Logan'

In [24]:
wwf_specs.columns

Index(['Imp. cal. factor ICF', 'Imp. san. factor ISF', 'Length factor LF',
       'Slope factor SF', 'RDII Factor RF', 'Umax (mm)', 'Lmax (mm)',
       'Overland coeff. Cqof', 'Groundwater coefficient GwCarea',
       'TC overland flow (h)', 'TC interflow (h)', 'TC baseflow (h)',
       'Threshold overland flow', 'Threshold interflow', 'Threshold GW flow',
       'Specific groundwater yield', 'Minimum groundwater depth',
       'Maximum groundwater depth causing baseflow',
       'Groundwater depth for unit capilary flux', 'Number sanitary',
       'Number combined', 'Number stormwater', 'Total number of catchments',
       'Area sanitary (ha)', 'Area combined (ha)', 'Area stormwater (ha)',
       'Drainage area (ha)', 'RDII Area (ha)', 'Steep Impervious area (ha)',
       'Flat Impervious area (ha)', 'Total', 'Average Length (m)',
       'Average slope (m/m)'],
      dtype='object')

In [70]:
error_list = []
for gauge in gauge_ids:
    
#      if gauge == 'GE1':
        

        location_type = gauges.loc[gauge,'Location Type']
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName']) 
        number_catchments = wwf_specs.loc[zone,'Number sanitary'] \
            + wwf_specs.loc[zone,'Number combined'] \
            + wwf_specs.loc[zone,'Number stormwater']
        
        effluent_types = []
        if wwf_specs.loc[zone,'Number sanitary'] > 0:
            effluent_types.append('Sanitary')
        if wwf_specs.loc[zone,'Number combined'] > 0:
            effluent_types.append('Combined')
        if wwf_specs.loc[zone,'Number stormwater'] > 0:
            effluent_types.append('Storm')
        
        for i, effluent_type in enumerate(effluent_types):
            if i == 0:
                effluent_type_string = effluent_type
            elif i == len(effluent_types)-1:
                effluent_type_string += ' and ' + effluent_type
            else:
                effluent_type_string += ', ' + effluent_type          
            
            

        
        local_population = 0
        for ww_type in ww_types:
            if ww_type in res_types:                
                local_population += dwf_specs.loc[zone,ww_type + '_Population']
                
        
        total_area = round(wwf_specs.loc[zone,'Drainage area (ha)'],1)

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
#---------------------------------------------------------------------------------------------------------------------        
 
    
            f.write('* {\n')
            f.write('  box-sizing: border-box;\n')
            f.write('}\n')

            f.write('.row {\n')
            f.write('  margin-left:-5px;\n')
            f.write('  margin-right:-5px;\n')

            f.write('}\n')

            f.write('.column {\n')
            f.write('  float: left;\n')
            f.write('  width: 20%;\n')
            f.write('  padding: 5px;\n')
            f.write('}\n')

            f.write('/* Clearfix (clear floats) */\n')
            f.write('.row::after {\n')
            f.write('  content: "";\n')
            f.write('  clear: both;\n')
            f.write('  display: table;\n')
            f.write('}\n')

            f.write('table.first, table.first th, table.first td {\n')
            f.write('  border: 2px solid;\n')
            f.write('  padding: 4px;\n')

            f.write('  border-collapse: collapse;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 800px;\n')
            f.write('}\n') 

            f.write('table.second {\n')
            f.write('  border: 2px solid;\n')
            f.write('  height: 35px;\n')
            f.write('  width: 350px;\n')

            f.write('})\n')    
    
#---------------------------------------------------------------------------------------------------------------------        
             
#             f.write('table.grid, th, td {')
#             f.write('  border: 1px solid;')
#             f.write('  padding: 4px;')
#             f.write('  border-collapse: collapse;')
            
            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>Number of Catchments</td>\n')
            f.write('<td>'+ str(number_catchments) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Zone Area (Ha)</td>\n')
            f.write('<td>'+ str(total_area) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Local Residential Population</td>\n')
            f.write('<td>'+ str(int(local_population)) +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>Effluent Type</td>\n')
            f.write('<td>'+ effluent_type_string +'</td>\n')
            f.write('</tr>\n')            
            for ww_type in ww_types:
                if ww_type not in res_types:
                    f.write('<tr>\n')
                    f.write('<td>' + ww_type + ' Area (Ha)</td>\n')
                    f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_' + description],1)) +'</td>\n')
                    f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('</table>')
            
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table class="first">\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table class="first">\n')
            f.write('  <tr>\n')
            f.write('    <th>Simulation</th>\n')
            f.write('    <th>Start Date</th>\n')
            f.write('    <th>End Date</th>\n')
            f.write('    <th>Duration</th>\n')
            f.write('  </tr>\n')

            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                duration = str((end - start).days) + ' days'
                f.write('  <tr>\n')
                f.write('    <td>' + description + '</th>\n')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                f.write('    <td>' + duration + '</th>\n')
                f.write('  </tr>\n')                   
            f.write('</table>\n')
                                      
            f.write('<h2>4. Zone Map</h2>\n')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')
            f.write('<h2>5. DWF Calibration Summary</h2>\n')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
            f.write('<h2>6. WWF Calibration and Validation Summary</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
            f.write('<h2>7. Calibration Issues</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
            f.write('<h2>8. Recommendations</h2>\n')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')
            f.write('<h2>9. DWF Calibration Plots and Statistics</h2>\n')

            wwf_plot_header_added = False
            for period_spec in period_specs:

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>10. WWF Calibration Plots</h2>\n')
                    wwf_plot_header_added = True
                                    

                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                
                if location_type == 'PS':
                    measured_gauge['Discharge_Hourly'] = measured_gauge['Flow'].rolling('1h').mean()
                
                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                
                if location_type == 'PS':
                    result_flow_gauge['Discharge_Hourly'] = result_flow_gauge['Discharge'].rolling('1h').mean()
                
                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]
                #---------------------------------------------------------------------------------------------
                compare_stats = []
                peak_level_model = round(result_level_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Model',peak_level_model,'m'])
                peak_level_measured = round(measured_gauge.Level.max(),2)
                compare_stats.append(['Peak Level Measured',peak_level_measured,'m'])
                compare_stats.append(['Peak Level Difference',round(peak_level_model - peak_level_measured,2),'m'])
                volume_model = round(result_flow_gauge.Volume.sum(),0).item()
                compare_stats.append(['Volume Model',volume_model,'m3'])
                volume_measured = round(measured_gauge.Volume.sum(),0).item() #convert to native float to avoid inf
                compare_stats.append(['Volume Measured',volume_measured,'m3'])
                try:
                    compare_stats.append(['Volume Difference',round((volume_model - volume_measured) / volume_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Volume Difference','','%'])
                peak_flow_model = round(result_flow_gauge.Discharge.max(),1)
                compare_stats.append(['Peak Flow Model',peak_flow_model,'L/s'])
                peak_flow_measured = round(measured_gauge.Flow.max(),1)
                compare_stats.append(['Peak Flow Measured',peak_flow_measured,'L/s'])
                try:
                    compare_stats.append(['Peak Flow Difference',round((peak_flow_model - peak_flow_measured) / peak_flow_measured * 100, 0),'%'])
                except:
                    compare_stats.append(['Peak Flow Difference','','%'])
                
                
                f.write('<h3>' + chart_header + '</h3>\n')  
                #----------------------------------------------------------------------------------------------------   
                if period_spec[:3] == "DWF":
                    
                    f.write('<div class="row">')
                    
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Description</th>\n')
                    f.write('    <th>Value</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')                   
                    
                    #sim and measured stats
                    for compare_stat in compare_stats:
                        f.write('  <tr>\n')
                        f.write('    <td>' + compare_stat[0] + '</th>\n')
                        f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                        f.write('    <td>' + compare_stat[2] + '</th>\n')
                        f.write('  </tr>\n')
                        
                    f.write('</table>\n')
                    f.write('</div>')                    
                    
                    #Loading rate
                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Loading Rate</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')
                                       
                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 3
                        else: 
                            decimals = 1
                        rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                        unit = all_types.loc[ww_type,'Unit2']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(rate) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')

                    #Population and ICI areas
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    f.write('    <th>Total</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for ww_type in ww_types:
                        if ww_type in res_types:
                            decimals = 0
                        else: 
                            decimals = 1
                        description = unit = all_types.loc[ww_type,'Description']
                        local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
                        total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
                        unit = all_types.loc[ww_type,'Unit1']

                        f.write('  <tr>\n')
                        f.write('    <td>' + ww_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        f.write('    <td>' + str(total_val) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')

                    f.write('<div class="column">')
                    #Waterload
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Load Type</th>\n')
                    f.write('    <th>Local</th>\n')
                    f.write('    <th>Total</th>\n')
                    f.write('    <th>Unit</th>\n')
                    f.write('  </tr>\n')

                    for load_type in all_types.index.values.tolist():

                        local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
                        total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
                        unit = 'm3/d'

                        f.write('  <tr>\n')
                        f.write('    <td>' + load_type + '</th>\n')
                        f.write('    <td>' + str(local_val) + '</th>\n')
                        f.write('    <td>' + str(total_val) + '</th>\n')
                        f.write('    <td>' + unit + '</th>\n')
                        f.write('  </tr>\n') 
                        
                    f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')

                    f.write('</table>\n')
                    f.write('</div>')
                    f.write('</div>')
                    
                else:

                    total_rows = 10
                    f.write('<div class="row">')
                    for i in range(0,4): #Cycle through the three table types.   
                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Description</th>\n')
                        f.write('    <th>Value</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        current_row = 0
                        if i == 0:
                            for compare_stat in compare_stats:
                                f.write('  <tr>\n')
                                f.write('    <td>' + compare_stat[0] + '</th>\n')
                                f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                                f.write('    <td>' + compare_stat[2] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1                
                        else:
                            for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

                                decimals = 2

                                rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                                unit = all_types.loc[ww_type,'Unit2']


                                f.write('  <tr>\n')
                                f.write('    <td>' + row['Description'] + '</th>\n')
                                if row['DF'] == 'wwf_specs':
                                    f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],2)) + '</th>\n')
                                else: 
                                    f.write('    <td>' + str(round(measured_rain[rain_gauge].sum(),1)) + '</th>\n')
                                f.write('    <td>' + row['Unit'] + '</th>\n')
                                f.write('  </tr>\n')
                                current_row += 1

                        remaining_rows = total_rows - current_row

                        for remaining_row in range(remaining_rows):
                            f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')

                    f.write('</div>')
                    #----------------------------------------------------------------------------------------------


                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                    'Dicharge at Gauge ' + gauge + ', Reach ' + pipe + ', Rain Gauge ' + rain_gauge,  
                                                    'Water Level at Gauge ' + gauge + ', Node ' + node),
                                                    specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                   horizontal_spacing = 0.1)

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                if location_type == 'PS':
                    fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge_Hourly, mode='lines',name='Model Flow Hourly',line_color='yellow'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Discharge_Hourly, mode='lines',name='Measured Flow Hourly',line_color='black'),1,1)
                
                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level',line_color='green'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level',line_color='purple'),1,2)
                
                                
                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=300,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })

                
                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
            #     fig['layout']['yaxis2']['autorange']='reversed'
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'



            #     fig.update_yaxes(rangemode="tozero")    

#                 fig.show()
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                f.write('<br>')



        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for FA2
Writing html for GN14
Writing html for HL14
Writing html for HL18
Writing html for HL1
Writing html for HL33
Writing html for HL94
Writing html for HL95
Writing html for HL97
Writing html for HL9
Writing html for MK1
Writing html for MK20
Writing html for MK3
Writing html for NV13
Writing html for NV19
Writing html for NV1
Writing html for NV20
Writing html for NV21
Writing html for NV31
Writing html for NV45
Writing html for NV8
Writing html for NV93
Writing html for NV94
Writing html for NV9
Writing html for NVLS1
Writing html for NVLS5-450
Writing html for NVLS5-500
Writing html for NVLS5-Blowdown
Writing html for NVLS9
Writing html for TW3B-MP1
Writing html for TW3B-MP2
Writing html for GE1
Writing html for GE2
Writing html for GE3
Writing html for GE4
Writing html for GE5
Writing html for LY1
Writing html for LY2
Writing html for LY7


In [61]:
result_flow_gauge

,ResultFile,MUID,DateTimeRef,Discharge,Volume,Discharge_Hourly
2019-12-29 00:00:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2019-12-29 00:00:00,4.302546e-28,1.290764e-28,4.302546e-28
2019-12-29 00:05:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2019-12-29 00:05:00,0.000000e+00,0.000000e+00,2.151273e-28
2019-12-29 00:10:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2019-12-29 00:10:00,0.000000e+00,0.000000e+00,1.434182e-28
2019-12-29 00:15:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2019-12-29 00:15:00,2.751222e+01,8.253666e+00,6.878055e+00
2019-12-29 00:20:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2019-12-29 00:20:00,0.000000e+00,0.000000e+00,5.502444e+00
...,...,...,...,...,...,...
2020-01-01 23:40:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2020-01-01 23:40:00,0.000000e+00,0.000000e+00,7.070063e+00
2020-01-01 23:45:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2020-01-01 23:45:00,0.000000e+00,0.000000e+00,7.036135e+00
2020-01-01 23:50:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2020-01-01 23:50:00,3.038234e+01,9.114702e+00,9.567997e+00
2020-01-01 23:55:00,NSSA_WWF_Cal_2019-12-29_Base.res1d,50924,2020-01-01 23:55:00,2.721311e+01,8.163933e+00,1.183576e+01


In [ ]:
# type(np.float32(volume_measured).item())
type(volume_measured.item())

In [501]:
type(volume_model)

numpy.float64

In [507]:
type(peak_flow_model)

float

In [57]:
gauge_ids

['FA2',
 'GN14',
 'HL14',
 'HL18',
 'HL1',
 'HL33',
 'HL94',
 'HL95',
 'HL97',
 'HL9',
 'MK1',
 'MK20',
 'MK3',
 'NV13',
 'NV19',
 'NV1',
 'NV20',
 'NV21',
 'NV31',
 'NV45',
 'NV8',
 'NV93',
 'NV94',
 'NV9',
 'NVLS1',
 'NVLS5-450',
 'NVLS5-500',
 'NVLS5-Blowdown',
 'NVLS9',
 'TW3B-MP1',
 'TW3B-MP2',
 'GE1',
 'GE2',
 'GE3',
 'GE4',
 'GE5',
 'LY1',
 'LY2',
 'LY7']

In [509]:
gauges_test = gauges.copy()

In [510]:
gauges_test

,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
Gauge,,,,,,,,,,,,,,,,,
FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node


In [51]:
gauges

,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
Gauge,,,,,,,,,,,,,,,,,
FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node


In [73]:
dwf_specs.columns

Index(['Mixed_Rate', 'ResHD_Rate', 'ResLD_Rate', 'Commercial_Rate',
       'Industrial_Rate', 'Institutional_Rate', 'Mixed_Population',
       'ResHD_Population', 'ResLD_Population', 'Commercial_Area',
       'Industrial_Area', 'Institutional_Area', 'Mixed_WaterLoad',
       'ResHD_WaterLoad', 'ResLD_WaterLoad', 'Commercial_WaterLoad',
       'Industrial_WaterLoad', 'Institutional_WaterLoad', 'Baseflow_WaterLoad',
       'Total_WaterLoad', 'Mixed_Population_Upstream',
       'ResHD_Population_Upstream', 'ResLD_Population_Upstream',
       'Commercial_Area_Upstream', 'Industrial_Area_Upstream',
       'Institutional_Area_Upstream', 'Mixed_WaterLoad_Upstream',
       'ResHD_WaterLoad_Upstream', 'ResLD_WaterLoad_Upstream',
       'Commercial_WaterLoad_Upstream', 'Industrial_WaterLoad_Upstream',
       'Institutional_WaterLoad_Upstream', 'Baseflow_WaterLoad_Upstream',
       'Total_WaterLoad_Upstream'],
      dtype='object')